In [18]:
# Import block
import os
import pandas as pd
import numpy as np
import time
from math import sqrt
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pandas.plotting import scatter_matrix
from importlib import reload
from sklearn.feature_selection import VarianceThreshold
from collections import Counter
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

Unable to revert mtime: /Library/Fonts
Fontconfig warning: ignoring UTF-8: not a valid region tag


In [44]:
# Initialize pandas dataframe

cols_to_keep = [
    'tBodyAcc-mean()-X',
    'tBodyAcc-mean()-Y',
    'tBodyAcc-mean()-Z',
    'tBodyGyro-mean()-X',
    'tBodyGyro-mean()-Y',
    'tBodyGyro-mean()-Z',
    'Activity',
]

TRAIN_DATASET_PATH = './Dataset/train.csv'
TEST_DATASET_PATH = './Dataset/test.csv'

train_df = pd.read_csv(TRAIN_DATASET_PATH, index_col=False)
test_df = pd.read_csv(TEST_DATASET_PATH, index_col=False)

train_df = train_df[cols_to_keep]
test_df = test_df[cols_to_keep]

train_df

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,Activity
0,0.288585,-0.020294,-0.132905,-0.006101,-0.031365,0.107725,STANDING
1,0.278419,-0.016411,-0.123520,-0.016112,-0.083894,0.100584,STANDING
2,0.279653,-0.019467,-0.113462,-0.031698,-0.102335,0.096127,STANDING
3,0.279174,-0.026201,-0.123283,-0.043410,-0.091386,0.085538,STANDING
4,0.276629,-0.016570,-0.115362,-0.033960,-0.074708,0.077392,STANDING
...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.035024,-0.093011,0.124412,WALKING_UPSTAIRS
7348,0.273853,-0.007749,-0.147468,0.118696,-0.095746,0.033277,WALKING_UPSTAIRS
7349,0.273387,-0.017011,-0.045022,-0.213192,0.039321,0.197982,WALKING_UPSTAIRS
7350,0.289654,-0.018843,-0.158281,-0.406205,0.068797,0.177467,WALKING_UPSTAIRS


In [49]:
# Split dataframe into features and labels

def split_df(df):
  features = df.drop('Activity', axis=1)
  labels = df['Activity']
  return features, labels

train_features_df, train_labels_df = split_df(train_df)
test_features_df, test_labels_df = split_df(train_df)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z
0,0.288585,-0.020294,-0.132905,-0.006101,-0.031365,0.107725
1,0.278419,-0.016411,-0.123520,-0.016112,-0.083894,0.100584
2,0.279653,-0.019467,-0.113462,-0.031698,-0.102335,0.096127
3,0.279174,-0.026201,-0.123283,-0.043410,-0.091386,0.085538
4,0.276629,-0.016570,-0.115362,-0.033960,-0.074708,0.077392
...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.035024,-0.093011,0.124412
7348,0.273853,-0.007749,-0.147468,0.118696,-0.095746,0.033277
7349,0.273387,-0.017011,-0.045022,-0.213192,0.039321,0.197982
7350,0.289654,-0.018843,-0.158281,-0.406205,0.068797,0.177467


In [68]:
# Implement sliding window

def process_df_to_sliding_window(df):
    df_out = df.iloc[[0]]
    for i in range(20):
        df_out = pd.concat(df, df2], axis=1)
        df_out.columns = list(range(len(df_out.columns)))
        df_out

process_df_to_sliding_window(train_features_df)

   tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0           0.288585          -0.020294          -0.132905   

   tBodyGyro-mean()-X  tBodyGyro-mean()-Y  tBodyGyro-mean()-Z  
0           -0.006675           -0.031365            0.107725  


In [62]:
# Normalize and pre-process data

from sklearn.preprocessing import normalize

def normalize_data(df):
    for col in df.columns:
        df[col] = df[col] / df[col].abs().max()
    
    return df
    
train_features_df = normalize_data(train_features_df)
test_features_df = normalize_data(test_features_df)

In [63]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes = (10,50,4))